Voglio modellare un solo pullman con 10 posti e clienti che arrivano ogni 1 tu.

1) Il bus è in attesa e appena salgono 10 cienti parte.
2) impiega 12 tu ad andare/scaricare e tornare.
3) Carica altri 10 clienti e riparte.

In [68]:
import simpy

# CODICE FUNZIONANTE 

Perchè funziona: Prima facevo la get subito all'inizio del processo navetta (bus), ma questo permetteva agli altri bus di mettersi dentro lo store.

In questo caso la get viene fatta solo quando il bus è pieno e l'autobus si sveglia ogni poco per controllare che ci siano viaggiatori pronti a salire, quando è pieno parte e libera lo store.

In [72]:
import simpy
import random

CAPACITA_NAVETTA = 5  # Numero di passeggeri che una navetta può trasportare
TEMPO_VIAGGIO = 5    # Tempo di viaggio in minuti
INTERVALLO_ARRIVO_CLIENTI = 2 # Tempo medio tra un cliente e l'altro

class Navetta:
    def __init__(self, env, name, capacita):
        self.env = env
        self.name = name
        self.capacita = capacita
        self.passeggeri_a_bordo = 0
        self.in_viaggio = False
        # Questo Store interno alla navetta serve per i passeggeri che DEVONO salire su QUESTA navetta
        self.coda_imbarco_interna = simpy.Store(env) 

def generatore_clienti(env, coda_passeggeri_generale, tempo_massimo_simulazione):
    i = 0
    while True:
        yield env.timeout(5)
        i += 1
        print(f'{env.now:.2f}: Cliente {i} arriva e si mette in coda generale.')
        # I clienti si mettono in una coda generale, non cercano un bus specifico subito
        coda_passeggeri_generale.put(f'Cliente_{i}') 
        if env.now > tempo_massimo_simulazione: 
            break

def cliente(env, name, navetta_destinazione):
    # Il cliente ora sa quale navetta deve salire e aspetta di essere imbarcato
    print(f'{env.now:.2f}: {name} sta salendo sulla {navetta_destinazione.name}.')
    yield env.timeout(0.5) # Tempo per salire
    print(f'{env.now:.2f}: {name} è salito sulla {navetta_destinazione.name}.')
    # L'azione del cliente finisce qui per questa simulazione, ma potrebbe includere il viaggio e lo sbarco

def processo_navetta(env, navetta_obj, bus_attivi_per_imbarco, coda_passeggeri_generale):
    # Ogni navetta inizia il suo ciclo
    while True:
        # La navetta si mette in coda per essere la prossima ad attivarsi per l'imbarco
        yield bus_attivi_per_imbarco.put(navetta_obj) 
        print(f'\n{env.now:.2f}: {navetta_obj.name} è alla fermata e attiva per l\'imbarco.')
        
        navetta_obj.passeggeri_a_bordo = 0

        # La navetta ora attende di riempirsi dalla coda generale
        while navetta_obj.passeggeri_a_bordo < navetta_obj.capacita:
            # Tenta di prendere un passeggero dalla coda generale
            try:
                # Aspetta che un passeggero sia disponibile nella coda generale
                nome_passeggero = yield coda_passeggeri_generale.get() 
                
                # Una volta che un passeggero è disponibile, lo "imbarca" su QUESTA navetta
                env.process(cliente(env, nome_passeggero, navetta_obj))
                navetta_obj.passeggeri_a_bordo += 1
                
                print(f'{env.now:.2f}: {navetta_obj.name}: Imbarcato {nome_passeggero}. Totale: {navetta_obj.passeggeri_a_bordo}/{navetta_obj.capacita}.')
                
            except simpy.Empty as e:
                # Se non ci sono passeggeri, la navetta aspetta che ne arrivino
                print(f'{env.now:.2f}: {navetta_obj.name} aspetta passeggeri...')
                yield env.timeout(1) # Attende un po' e riprova

        print(f'{env.now:.2f}: {navetta_obj.name} è PIENA ({navetta_obj.passeggeri_a_bordo} passeggeri). Parte ora!')
        
        # Una volta piena, la navetta "si rimuove" dallo store dei bus attivi per l'imbarco
        # Questo permette alla prossima navetta in coda di attivarsi.
        yield bus_attivi_per_imbarco.get() # La navetta si rimuove dallo stato "attiva per imbarco"

        # Simula il viaggio
        navetta_obj.in_viaggio = True
        yield env.timeout(TEMPO_VIAGGIO)
        navetta_obj.in_viaggio = False
        
        print(f'{env.now:.2f}: {navetta_obj.name} è arrivata a destinazione e scarica i passeggeri. Ora torna vuota.')
        navetta_obj.passeggeri_a_bordo = 0 # Scarica i passeggeri
        # La navetta è ora di nuovo disponibile per il prossimo ciclo (tornerà in coda per l'imbarco)

# --- Configurazione della Simulazione ---
env = simpy.Environment()

# Questa è la coda dove i passeggeri si accodano appena arrivano
coda_passeggeri_generale = simpy.Store(env) 

# Questo Store gestisce quale navetta è ATTIVA per l'imbarco in un dato momento
# Ha capacità 1, quindi solo UNA navetta può essere messa qui alla volta.
# Le navette "competono" per essere qui.
bus_attivi_per_imbarco = simpy.Store(env, capacity=1)

NUM_NAVETTE_TOTALI = 3 

# Avvia i processi delle navette
for i in range(NUM_NAVETTE_TOTALI):
    navetta_obj = Navetta(env, f'Bus_{i+1}', CAPACITA_NAVETTA)
    env.process(processo_navetta(env, navetta_obj, bus_attivi_per_imbarco, coda_passeggeri_generale))

# Avvia il generatore di clienti
TEMPO_SIMULAZIONE = 50
env.process(generatore_clienti(env, coda_passeggeri_generale, TEMPO_SIMULAZIONE))

# Esegui la simulazione
print(f'--- Simulazione avviata per {TEMPO_SIMULAZIONE} minuti ---')
env.run(until=TEMPO_SIMULAZIONE)
print('--- Simulazione terminata ---')

--- Simulazione avviata per 50 minuti ---

0.00: Bus_1 è alla fermata e attiva per l'imbarco.
5.00: Cliente 1 arriva e si mette in coda generale.
5.00: Bus_1: Imbarcato Cliente_1. Totale: 1/5.
5.00: Cliente_1 sta salendo sulla Bus_1.
5.50: Cliente_1 è salito sulla Bus_1.
10.00: Cliente 2 arriva e si mette in coda generale.
10.00: Bus_1: Imbarcato Cliente_2. Totale: 2/5.
10.00: Cliente_2 sta salendo sulla Bus_1.
10.50: Cliente_2 è salito sulla Bus_1.
15.00: Cliente 3 arriva e si mette in coda generale.
15.00: Bus_1: Imbarcato Cliente_3. Totale: 3/5.
15.00: Cliente_3 sta salendo sulla Bus_1.
15.50: Cliente_3 è salito sulla Bus_1.
20.00: Cliente 4 arriva e si mette in coda generale.
20.00: Bus_1: Imbarcato Cliente_4. Totale: 4/5.
20.00: Cliente_4 sta salendo sulla Bus_1.
20.50: Cliente_4 è salito sulla Bus_1.
25.00: Cliente 5 arriva e si mette in coda generale.
25.00: Bus_1: Imbarcato Cliente_5. Totale: 5/5.
25.00: Bus_1 è PIENA (5 passeggeri). Parte ora!
25.00: Cliente_5 sta salendo sull